## This notebook will help you train a vanilla Point-Cloud AE with the basic architecture we used in our paper.
    (it assumes latent_3d_points is in the PYTHONPATH and the structural losses have been compiled)

In [1]:

import tflearn
import numpy as np

import os.path as osp
import tensorflow as tf
from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder, unpickle_data

from latent_3d_points.src.tf_utils import reset_tf_graph
import ipyvolume as ipv

/home/yz6/.virtualenvs/gan_env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Define Basic Parameters

In [3]:
top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.

# 
# model_dir = osp.join(top_out_dir, 'single_class_ae_chair_emd')
model_dir = osp.join(top_out_dir, 'single_class_ae_chair_chamfer')
experiment_name = 'single_class_ae_chair_chamfer'
n_pc_points = 2048                              # Number of points per model.
bneck_size = 128                                # Bottleneck-AE size
ae_loss = 'chamfer'                             # Loss to optimize: 'emd' or 'chamfer'
class_name = 'chair'
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)

train_dir = create_dir(osp.join(top_out_dir, experiment_name))
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            train_dir = train_dir,
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).
reset_tf_graph()
print tf.get_default_graph().as_graph_def()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

versions {
  producer: 24
}

Building Encoder
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
encoder_conv_layer_0 conv params =  256 bnorm params =  128
Tensor("single_class_ae_chair_chamfer_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_1 conv params =  8320 bnorm params =  256
Tensor("single_class_ae_chair_chamfer_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_2 conv params =  16512 bnorm params =  256
Tensor("single_class_ae_chair_chamfer_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  33024 bnorm params =  512
Tensor("single_class_ae_chair_chamfer_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  32896 bnorm params =  256
Tensor("single_class_ae_chair_chamfer_2/Relu_4:0", shape=(?, 2048, 128), dtype=float

Load Point-Clouds

In [4]:
ae.restore_model(model_dir, 100)



INFO:tensorflow:Restoring parameters from ../data/single_class_ae_chair_chamfer/models.ckpt-100


In [5]:
hidden = np.load('/home/yz6/code/latent_3d_points/data/lgan_simplegan_ae_chair_chamfer/hidden.npy')
hidden = hidden[:100]
fake = ae.decode(hidden)

In [16]:

obj = fake[6]
x, y, z = obj[:, 0], obj[:, 1], obj[:, 2]
ipv.xyzlim(-0.5, 0.5)
ipv.quickscatter(x + 0.5 , y + 0.5, z +0.5, size=0.8, marker="sphere", data_max = 0.5)

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white', dtype='|S5'), geo=u'sphere', size=array(0.8), size_selected=array(2.6), x=array([0.5455384 , 0.45088375, 0.4427423 , ..., 0.40980506, 0.43902504,
       0.4012595 ], dtype=float32), y=array([0.53536046, 0.5555244 , 0.35704795, ..., 0.5770031 , 0.5311581 ,
       0.62461853], dtype=float32), z=array([0.45858613, 0.4713546 , 0.3323913 , ..., 0.5518397 , 0.4807948 ,
       0.4236276 ], dtype=float32))], style={'box': {'visible': True}, 'axes': {'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}, 'label': {'color': 'black'}}, 'background-color': 'white'}, tf=None, width=400, xlim=[0.0, 1.0], ylim=[0.0, 1.0], zlim=[0.0, 1.0]),))

In [7]:
pc = list(unpickle_data('/home/yz6/code/latent_3d_points/data/single_class_ae_chair_chamfer/val_pc.pkl'))[0].point_clouds
pc = pc[:100]
recon, loss = ae.reconstruct(pc)



In [8]:
ipv.quickscatter(x + 0.5 , y + 0.5, z +0.5, size=0.8, marker="sphere", data_max = 0.5)
obj = recon[3]
x, y, z = obj[:, 0], obj[:, 1], obj[:, 2]
ipv.xyzlim(-0.5, 0.5)
ipv.quickscatter(x + 0.5 , y + 0.5, z +0.5, size=0.8, marker="sphere", data_max = 0.5)


VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white', dtype='|S5'), geo=u'sphere', size=array(0.8), size_selected=array(2.6), x=array([0.5726589 , 0.41959262, 0.3472005 , ..., 0.32238042, 0.37024742,
       0.34245506], dtype=float32), y=array([0.562664  , 0.60943925, 0.3554434 , ..., 0.60026324, 0.57444173,
       0.60189605], dtype=float32), z=array([0.36519974, 0.37066045, 0.23486286, ..., 0.5899884 , 0.44926208,
       0.35273868], dtype=float32))], style={'box': {'visible': True}, 'axes': {'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}, 'label': {'color': 'black'}}, 'background-color': 'white'}, tf=None, width=400, xlim=[0.0, 1.0], ylim=[0.0, 1.0], zlim=[0.0, 1.0]),))

In [17]:
ipv.quickscatter(x + 0.5 , y + 0.5, z +0.5, size=0.8, marker="sphere", data_max = 0.5)
obj = pc[3]
x, y, z = obj[:, 0], obj[:, 1], obj[:, 2]
ipv.xyzlim(-0.5, 0.5)
ipv.quickscatter(x + 0.5 , y + 0.5, z +0.5, size=0.8, marker="sphere", data_max = 0.5)


VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white', dtype='|S5'), geo=u'sphere', size=array(0.8), size_selected=array(2.6), x=array([0.66372305, 0.6641303 , 0.6528984 , ..., 0.20977607, 0.21498862,
       0.21341854], dtype=float32), y=array([0.33636367, 0.6644704 , 0.6661057 , ..., 0.46473402, 0.46727967,
       0.4895306 ], dtype=float32), z=array([0.18943274, 0.18945801, 0.19013146, ..., 0.808208  , 0.8084043 ,
       0.81056726], dtype=float32))], style={'box': {'visible': True}, 'axes': {'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}, 'label': {'color': 'black'}}, 'background-color': 'white'}, tf=None, width=400, xlim=[0.0, 1.0], ylim=[0.0, 1.0], zlim=[0.0, 1.0]),))